# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**: Liying Ma

**ID**: Lm698

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/BEE4750/hws/hw5-spicyfish-test`


In [3]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [8]:
# Create model
model = Model(HiGHS.Optimizer)

# Sets
cities = 1:3
facilities = 1:3 # 1=LF, 2=MRF, 3=WTE

# Parameters
S = [100, 90, 120] # Waste production at each city (Mg/day)
K = [200, 350, 210] # Capacity at each facility (Mg/day) [LF, MRF, WTE]
transportation_cost = 1.5 # $/Mg-km

# Distances between cities and facilities (km)
distances = [
    [5 30 15];   # City 1 to facilities
    [15 25 10];  # City 2 to facilities
    [13 45 20]  # City 3 to facilities
]

# Facility-to-facility distances (km)
facility_distances = [
    [0 32 18];   # From LF to others
    [32 0 15];   # From MRF to others
    [18 15 0]    # From WTE to others
]

# Fixed costs ($/day)
fixed_costs = [2000, 1500, 2500] # LF, MRF, WTE

# Variable costs ($/Mg)
variable_costs = [50, 7, 60] # Tipping costs

# Variables
@variable(model, W[cities, facilities] >= 0)  
@variable(model, R[facilities, facilities] >= 0) 
@variable(model, Y[facilities], Bin)            

# Objective function: Minimize total costs
@objective(model, Min, 
    # Transportation costs from cities to facilities
    sum(transportation_cost * distances[i,j] * W[i,j] for i in cities, j in facilities) +
    # Fixed costs
    sum(fixed_costs[j] * Y[j] for j in facilities) +
    # Variable costs for primary waste
    sum(variable_costs[j] * W[i,j] for i in cities, j in facilities) +
    # Transportation and handling costs for residuals
    sum(transportation_cost * facility_distances[k,j] * R[k,j] + variable_costs[j] * R[k,j] 
        for k in facilities, j in facilities if k != j)
)

# Constraints

# Mass balance at cities
for i in cities
    @constraint(model, sum(W[i,j] for j in facilities) == S[i])
end

# Capacity constraints at facilities
@constraint(model, W[1,3] + W[2,3] + W[3,3] + R[1,3] + R[2,3] <= K[3] * Y[3]) # LF
@constraint(model, W[1,2] + W[2,2] + W[3,2] <= K[2] * Y[2])          # MRF
@constraint(model, W[1,1] + W[2,1] + W[3,1] + R[2,1] <= K[1] * Y[1]) # WTE

# WTE ash generation
@constraint(model, R[1,3] == 
    0.16 * (W[1,1] + W[2,1] + W[3,1] + R[2,1]) +  # 16% ash from non-recycled waste
    0.14 * (0.4 * sum(W[i,2] for i in cities))     # 14% ash from recycled waste (40% of MRF input)
)

# MRF residual generation (60% of input goes to either WTE or LF)
@constraint(model, R[2,1] + R[2,3] == 0.6 * (W[1,2] + W[2,2] + W[3,2]))

# No self-loops in residual flows
for j in facilities
    @constraint(model, R[j,j] == 0)
end

# Big-M constraints for facility operation
M = sum(S) # Big M value based on total waste generation
for j in facilities
    @constraint(model, sum(W[i,j] for i in cities) + sum(R[k,j] for k in facilities if k != j) <= M * Y[j])
end

# Additional cost for recycled material at MRF
recycling_cost = 40 # $/Mg recycled
@expression(model, recycling_costs, 
    recycling_cost * 0.4 * sum(W[i,2] for i in cities))  # 40% recycling rate
@objective(model, Min, objective_function(model) + recycling_costs)

# Solve
optimize!(model)

# Print results
println("Objective value: \$", round(objective_value(model), digits=2), "/day")
println("\nWaste flows (Mg/day):")
for i in cities, j in facilities
    val = value(W[i,j])
    if val > 0.1 # Accounting for numerical precision
        println("City $i to Facility $j: ", round(val, digits=1))
    end
end

println("\nResidual flows (Mg/day):")
for k in facilities, j in facilities
    val = value(R[k,j])
    if val > 0.1 # Accounting for numerical precision
        println("Facility $k to Facility $j: ", round(val, digits=1))
    end
end

println("\nFacility operation status:")
for j in facilities
    println("Facility $j: ", value(Y[j]) > 0.5 ? "Operating" : "Not operating")
end

# Print recycling amount if MRF is operating
if value(Y[2]) > 0.5
    total_mrf_input = sum(value(W[i,2]) for i in cities)
    println("\nMRF recycling: ", round(0.4 * total_mrf_input, digits=1), " Mg/day")
end

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [6e-02, 4e+02]
  Cost   [3e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [9e+01, 1e+02]
Presolving model
11 rows, 15 cols, 52 nonzeros  0s
8 rows, 13 cols, 36 nonzeros  0s
8 rows, 13 cols, 36 nonzeros  0s

Solving MIP model with:
   8 rows
   13 cols (2 binary, 0 integer, 0 implied int., 11 continuous)
   36 nonzeros
MIP-Timing:     0.00018 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp 

### Answer
Objective value: $28534.0/day

- Waste flows (Mg/day):

    City 1 to Facility 1: 100.0

    City 2 to Facility 3: 90.0

    City 3 to Facility 1: 100.0

    City 3 to Facility 3: 20.0

- Residual flows (Mg/day):

    Facility 1 to Facility 3: 32.0

- Facility operation status:

    Facility 1: Operating

    Facility 2: Not operating

    Facility 3: Operating

The MRF (Facility 2) will not be used in the optimal solution. This makes sense because sending waste directly to LF and WTE facilities reduces transportation costs and avoids double-handling costs that would occur with MRF (where 60% of waste needs secondary treatment). The solution efficiently utilizes the capacities of both LF and WTE while minimizing total system costs.


## References

List any external references consulted, including classmates.